In [19]:
import pandas as pd
import missingno as msno
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt

In [20]:
df = pd.read_csv('./train_muestra_ct.csv')

In [6]:
df.head()
df.shape

(4049213, 10)

### Generar reporte automático



In [7]:
profile = ProfileReport(
    df,
    title="reporte",
    explorative=True,  
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "kendall": {"calculate": True},
        "phi_k": {"calculate": True},
    },
    missing_diagrams={
        "bar": True,
        "matrix": True,
        "heatmap": True,
    },
)

# Guardar como HTML
profile.to_file("reporte_df.html")
print("\nReporte generado")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:10<00:00,  1.02s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]


Reporte generado


In [17]:
df.describe()


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
count,3.954940e+06,3.954940e+06,3.954940e+06,3.954940e+06,3954940.0,3.954940e+06,3.954940e+06,3.954940e+06,3.954940e+06
mean,5.061126e+07,7.737407e+09,1.076650e+09,4.987972e+03,0.0,9.121728e+02,1.423909e+00,6.569088e-01,2.543485e+04
std,2.922227e+07,1.161138e+10,6.197073e+08,3.292600e+03,0.0,1.367284e+03,1.155937e+00,4.747417e-01,1.995633e+04
min,4.000000e+00,0.000000e+00,1.150000e+02,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.530224e+07,5.298234e+08,5.408046e+08,2.001000e+03,0.0,1.050000e+02,0.000000e+00,0.000000e+00,1.600000e+04
50%,5.060709e+07,2.691744e+09,1.071726e+09,4.979000e+03,0.0,3.850000e+02,1.000000e+00,1.000000e+00,2.100000e+04
75%,7.591725e+07,1.000152e+10,1.615726e+09,7.218000e+03,0.0,1.106000e+03,3.000000e+00,1.000000e+00,2.966600e+04
max,1.012303e+08,8.719290e+10,2.147482e+09,1.352200e+04,0.0,9.999000e+03,3.000000e+00,1.000000e+00,3.000000e+05


Verificar datos faltantes en cada columna

In [21]:
#print("Valores nulos por columna (TOTAL):\n", df.isnull().sum())
completitud = df.notna().mean() * 100
print("Porcentaje de completitud por columna:\n", completitud)

Porcentaje de completitud por columna:
 row_id                            100.000000
timestamp                         100.000000
user_id                           100.000000
content_id                        100.000000
content_type_id                   100.000000
task_container_id                 100.000000
user_answer                       100.000000
answered_correctly                100.000000
prior_question_elapsed_time        97.671819
prior_question_had_explanation     99.611431
dtype: float64


In [12]:
duplicados_exactos = df[df.duplicated(keep=False)]  

print(f"Número de registros duplicados exactos: {len(duplicados_exactos)}")
if not duplicados_exactos.empty:
    print("\nEjemplo de registros duplicados:")
    display(duplicados_exactos.sort_values(by=list(df.columns)).head())

Número de registros duplicados exactos: 0


In [14]:
print("valores interpretados como null")
print("Recuento de -1 en 'user_answer':", (df['user_answer'] == -1).sum())
print("Recuento de -1 en 'answered_correctly':", (df['answered_correctly'] == -1).sum())


valores interpretados como null
Recuento de -1 en 'user_answer': 0
Recuento de -1 en 'answered_correctly': 0


In [ ]:
# Detección de outliers en 'valor' (método IQR)
Q1 = df['timestamp'].quantile(0.25)
Q3 = df['timestamp'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['timestamp'] < (Q1 - 1.5*IQR)) | (df['timestamp'] > (Q3 + 1.5*IQR))]
print("\nNúmero de outliers en timestamp", len(outliers))



Número de outliers en timestamp 368842


<Axes: xlabel='timestamp'>

In [16]:
# Detección de outliers en 'valor' (método IQR)
Q1 = df['prior_question_elapsed_time'].quantile(0.25)
Q3 = df['prior_question_elapsed_time'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['prior_question_elapsed_time'] < (Q1 - 1.5*IQR)) | (df['prior_question_elapsed_time'] > (Q3 + 1.5*IQR))]
print("\nNúmero de outliers en prior_question_elapsed_time", len(outliers))



Número de outliers en prior_question_elapsed_time 288431


In [25]:
df['prior_question_elapsed_time_min'] = df['prior_question_elapsed_time'] / 60000
df['timestamp_hrs'] = df['timestamp'] / 3600000
df.head()


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,prior_question_elapsed_time_min,timestamp_min,timestamp_hrs
0,162814,2192514769,3459591,4406,0,155,0,0,11000.0,True,0.183333,609.031880,609.031880
1,32681,324855427,786638,7093,0,114,2,1,34400.0,True,0.573333,90.237619,90.237619
2,40971,335689585,1095916,9107,0,233,0,0,8000.0,True,0.133333,93.247107,93.247107
3,63321,2397825389,1409904,1053,0,362,1,1,16000.0,True,0.266667,666.062608,666.062608
4,186690,14882365329,3838215,3365,0,3255,2,1,27000.0,True,0.450000,4133.990369,4133.990369


### Conclusión sobre el análisis exploratorio:

Con la generación de nuestro reporte automático y algunas de las métricas básicas podemos realizar ciertas recomendaciones para nuestros datos.

Primero , debido al tipo de registro en nuestras variables: prior_question_elapsed_time y timestamp que se registraron en milisegundos son valores en extremo grandes y depende del modelo usado puede que convenga convertir estas variables a unidades mas acorde a.

Por otra parte en las variables de: prior_question_elapsed_time y prior_question_had_explanation existen registros con valores nulos y aunque representan muy pocos datos (menos de un 5%) comviene eliminarlos.

Por otra parte es conveniente deja las columnas que hacen referencia a los id de otras tablas puesto que para el modelo se necesitará juntarlas de acuerdo al tipo de preguntas.

Por último existen muchos outliers dentro de mis columnas numericas(relacionadas con el tiempo entre preguntas). Sin embargo debido a lo explicado comviene no eliminarlos

